In [1]:
import pandas as pd
from pycaret.regression import *

df = pd.read_csv("../../data/rumah123_cleaned.csv")

df = df.drop(columns=["Unnamed: 0", "name"], errors='ignore')
df

,price,location,bedroom,bathroom,land_area,building_area
0,1680,"Rungkut, Surabaya",3,3,67,120
1,950,"Medokan Ayu, Surabaya",3,3,72,140
2,500,"Wonorejo, Surabaya",2,1,30,36
3,950,"Sukolilo, Surabaya",2,2,50,55
4,600,"Kebraon, Surabaya",2,1,50,56
...,...,...,...,...,...,...
37353,730,Sidoarjo,3,1,144,120
37354,1200,"Sedati, Sidoarjo",3,2,90,112
37355,660,Sidoarjo,2,1,90,70
37356,600,"Sedati, Sidoarjo",2,1,84,60


In [2]:
df.duplicated().sum()

14417

In [3]:
df = df.drop_duplicates()

In [4]:
df.duplicated().sum()

0

In [ ]:
import ydata_profiling as pp

profile = pp.ProfileReport(df)
profile.to_file('Report_Rumah123.html')

In [6]:
setup(
    df, 
    target='price',
    transform_target=True, 
    normalize=True, 
    normalize_method='minmax',
    remove_outliers=True, 
    outliers_threshold=0.05,
    categorical_features=['location'],
    session_id=42
)

,Description,Value
0,Session id,42
1,Target,price
2,Target type,Regression
3,Original data shape,"(22941, 6)"
4,Transformed data shape,"(22138, 6)"
5,Transformed train set shape,"(15255, 6)"
6,Transformed test set shape,"(6883, 6)"
7,Numeric features,4
8,Categorical features,1
9,Preprocess,True


In [7]:
best = compare_models()

In [8]:
result = evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [8]:
tuned_model = tune_model(best, search_library='scikit-optimize', search_algorithm='bayesian')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,204.3913,72354.9339,268.9887,0.6697,0.2444,0.1905
1,211.8300,78469.3739,280.1239,0.6378,0.2602,0.2099
2,213.2902,83014.1209,288.1217,0.6444,0.2800,0.2249
3,212.7503,79353.6292,281.6978,0.6542,0.2590,0.2062
4,222.8607,83598.3306,289.1338,0.6356,0.2764,0.2263
5,209.4171,75736.7643,275.2031,0.6747,0.2525,0.2046
6,203.5958,71116.6978,266.6771,0.6905,0.2489,0.1989
7,214.5294,83781.0844,289.4496,0.6223,0.2753,0.2164
8,216.5194,83807.9552,289.4960,0.6268,0.2865,0.2304


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


In [9]:
holdout_pred = predict_model(tuned_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,209.6404,77218.5533,277.8823,0.6586,0.2616,0.2081


[LightGBM] [Warning] feature_fraction is set=0.7588282465072769, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7588282465072769
[LightGBM] [Warning] bagging_fraction is set=0.887437593014418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.887437593014418
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [10]:
new_data = df.copy()
new_data.drop('price', axis=1, inplace=True)
new_data.head()

,location,bedroom,bathroom,land_area,building_area
0,"Rungkut, Surabaya",3,3,67,120
1,"Medokan Ayu, Surabaya",3,3,72,140
2,"Wonorejo, Surabaya",2,1,30,36
3,"Sukolilo, Surabaya",2,2,50,55
4,"Kebraon, Surabaya",2,1,50,56


In [11]:
predictions = predict_model(tuned_model, data = new_data)
predictions.head()

[LightGBM] [Warning] feature_fraction is set=0.7588282465072769, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7588282465072769
[LightGBM] [Warning] bagging_fraction is set=0.887437593014418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.887437593014418
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


,location,bedroom,bathroom,land_area,building_area,prediction_label
0,"Rungkut, Surabaya",3,3,67,120,1263.256200
1,"Medokan Ayu, Surabaya",3,3,72,140,1205.837301
2,"Wonorejo, Surabaya",2,1,30,36,408.374002
3,"Sukolilo, Surabaya",2,2,50,55,916.593588
4,"Kebraon, Surabaya",2,1,50,56,652.589755


In [12]:
# save_model(tuned_model, 'house-price-rf')